In [1]:
from preprocessing import *
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import tensorflow as tf

2024-03-27 19:32:21.442526: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
parent_dir = './Data/In-lab/'
paths = sorted(list(os.listdir(parent_dir)))
# paths = ['P2']
data = []
interval_duration = 0.5
vec_size = int(15_000 * interval_duration)

for path in tqdm(paths):
    temp = get_data(parent_dir+path, interval_duration=interval_duration)
    dataX = temp[:, :vec_size].astype(float)    
    remaining = temp[:, vec_size:]

    dataX -= dataX.mean()
    dataX /= np.std(dataX)
    temp = np.concatenate([dataX, remaining], axis=1)
    data.append(temp)
data = np.concatenate(data, axis=0)

100%|██████████| 16/16 [00:07<00:00,  2.10it/s]


In [3]:
data.shape

(3274, 7516)

In [4]:
dataX = data[:, :vec_size].astype(float)
dataY = data[:, -1].astype(float)

In [5]:
from sklearn.model_selection import train_test_split

trainX, valX, trainY, valY = train_test_split(dataX, dataY)

In [6]:
trainX.shape

(2455, 7500)

plt.figure(figsize=(20, 6))
plt.plot(np.arange(trainX[0, :1000].size), trainX[0, :1000])
plt.show()

In [7]:
from model import get_model
import tensorflow.keras as keras


In [8]:
model = get_model(input_size=vec_size)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'],
              )
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 7500, 1)          4         
 ormalization)                                                   
                                                                 
 conv1d (Conv1D)             (None, 7498, 4)           16        
                                                                 
 conv1d_1 (Conv1D)           (None, 7496, 8)           104       
                                                                 
 conv1d_2 (Conv1D)           (None, 7494, 16)          400       
                                                                 
 batch_normalization_1 (Batc  (None, 7494, 16)         64        
 hNormalization)                                                 
                                                                 
 conv1d_3 (Conv1D)           (None, 7492, 32)          1

2024-03-27 19:32:31.394024: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
from sklearn.utils import class_weight

weights = class_weight.compute_class_weight('balanced',
                                            classes=np.unique(trainY),
                                            y=trainY)
weights = {
    0: weights[0],
    1: weights[1]
}


In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=5, min_lr=1e-7, min_delta=1e-4)

history = model.fit(trainX, trainY, validation_data=(valX, valY), verbose=1, epochs=100, class_weight=weights)#, callbacks=[reduce_lr])

Epoch 1/100
77/77 [==============================] - 37s 458ms/step - loss: 0.6718 - accuracy: 0.5507 - val_loss: 0.7001 - val_accuracy: 0.3907
Epoch 2/100
77/77 [==============================] - 35s 459ms/step - loss: 0.6634 - accuracy: 0.5393 - val_loss: 0.6866 - val_accuracy: 0.4884
Epoch 3/100
77/77 [==============================] - 35s 461ms/step - loss: 0.6581 - accuracy: 0.5462 - val_loss: 0.6908 - val_accuracy: 0.4737
Epoch 4/100
77/77 [==============================] - 36s 463ms/step - loss: 0.6628 - accuracy: 0.5756 - val_loss: 0.6784 - val_accuracy: 0.5238
Epoch 5/100
77/77 [==============================] - 36s 462ms/step - loss: 0.6520 - accuracy: 0.5703 - val_loss: 0.7000 - val_accuracy: 0.4847
Epoch 6/100
77/77 [==============================] - 36s 466ms/step - loss: 0.6534 - accuracy: 0.5597 - val_loss: 0.6886 - val_accuracy: 0.4628
Epoch 7/100
77/77 [==============================] - 36s 470ms/step - loss: 0.6536 - accuracy: 0.5804 - val_loss: 0.6981 - val_accuracy:

In [ ]:
model.predict(trainX[:10]), trainY[:10]

In [ ]:
plt.plot(np.arange(1500), trainX[1, :1500])
plt.show()

In [ ]:
plt.plot(np.arange(200), valX[1, :200])
plt.show()